# Sparse tensors

A new functionality of Tensor Fox is the support for sparse tensors. Let $T \in \mathbb{R}^{I_1 \times \ldots \times I_L}$ ba a sparse tensor with *nnz* nonzero entries. The tensor is represented as a triple *[data, idxs, dims]*, where *data* is an array of size $nnz$ such that *data[i]* is the $i$-th nonzero entry of $T$, with corresponding index *idxs[i]*. It is necessary to pass *idxs* also as an array, which will be of shape $nnz \times L$. Finally, we have that *dims* $= [I_1, \ldots, I_L]$. 

Below we create a fourth order sparse tensor $10 \times 10 \times 10 \times 10$ with only 6 nonzero entries. These entries are random and are located in random places of $T$. 

In [1]:
import numpy as np
import TensorFox as tfx
import matplotlib.pyplot as plt

## Small example

In [2]:
# Initial variables.
nnz = 6
n = 10
dims = (n, n, n, n)
L = len(dims)

# Create the nonzero values of T.
data = np.random.randn(nnz)

# Create indexes.
idxs = np.zeros((nnz, L), dtype=np.int64) 
for l in range(L):
    idxs[:, l] = np.random.randint(0, dims[l], size=(nnz))

# Define sparse tensor.
T = [data, idxs, dims]

Here we prefer the damped Gauss-Newton method to compute the CPD, but the Tensor Train CPD also works. We remark that it is not possible to use $\verb|tol| \_ \verb|mlsvd| = 0$ or $-1$ since these options doesn't perform compression and to work with large tensors as $T$ we must always to compress. In the case the user sets $\verb|display| = 3$ or $4$, the program computes only the error associated with the nonzero entries, otherwise we would face memory issues. 

In [3]:
class options:
    display = 1
    method = 'dGN'
    
R = 6
factors, output = tfx.cpd(T, R, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compression detected
    Compressing from (10, 10, 10, 10) to (4, 4, 5, 4)
-----------------------------------------------------------------------------------------------
Type of initialization: random
-----------------------------------------------------------------------------------------------
Computing CPD
Final results
    Number of steps = 16
    Relative error = 0.10687570830242643
    Accuracy =  89.31243 %


The relative error showed above only take in account the nonzero entries of $T$ (the same is valid for the compression error which is equal to zero in this example). The factor matrices of the decomposition may introduce small errors when approximating the zeros, and this small errors summed together does increase the actual error of the CPD. Thus the relative error showed above is a lower bound to the actual error, but usually it is close enough.

Since this is a small example we can put everything in dense format and verify what is the actual error. However this won't be possible for really large tensors. Regardless, the main point of the CPD is to approximate the nonzero entries, which is done.

In [4]:
# Generate the coordinate (dense) format of the approximation.
T_approx = tfx.cpd2tens(factors)

# Generate the dense format from the sparse representation.
T_dense = tfx.sparse2dense(data, idxs, dims)

# Compute the error.
np.linalg.norm(T_dense - T_approx)/np.linalg.norm(T_dense)

0.10687681769528798

## Memory cost

There is a big reduction in memory cost when working with sparse representations. Below we show a graph with the maximum memory cost attained in the computation of the CPDs of sparse $n \times n \times n$ tensors (blue curve) vs. the cost to store these tensors in dense format. As we can see, the difference is substantial, Tensor Fox does avoid the intermediate memory explosion problem. For instance, the sparse approach requires $3648$ megabytes when $n = 30000$, whereas the dense approach requires approximately $205$ terabytes. 

![sparse](sparse.png)

## Big example

Instead of generating random coordinates of the tensor to be nonzero, we can generate sparse factor matrices and then the associated sparse tensor. For this all we need is to call the function *gen_rand_sparse_tensor* with inputs **dims**, **R**, **nnz** as showed below. In this example we consider a fourth order tensor with dimensions $100 \times 100 \times 100 \times 100$, rank $R = 5$ and $nnz = 40$ nonzero entries in each factor matrix. We remark that the sparsity of the tensor is not necessarily equal to the sparsity of its factor matrices.

In [5]:
# Rank of the tensor.
R = 5

# Dimensions of the tensor.
dims = [100, 100, 100, 100]

# Number of nonzero entries of each factor matrix. 
nnz = 40

# Generate sparse tensor.
print('Factor matrix nonzero entries =', round(100 * nnz / np.prod(dims[0] * R), 2), '%')
data, idxs, dims, factors = tfx.gen_rand_sparse_tensor(dims, R, nnz)
print('Tensor nonzero entries =', round(100 * len(data) / np.prod(dims), 2), '%')

Factor matrix nonzero entries = 8.0 %
Tensor nonzero entries = 0.92 %


In this case we know the rank in advance so it is not necessary to guess. Note how the CPD is able to find a good approximation with less effort now.

In [6]:
# Define sparse tensor.
T = [data, idxs, dims]

options.display = 1
factors, output = tfx.cpd(T, R, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compression detected
    Compressing from [100 100 100 100] to (5, 5, 5, 5)
-----------------------------------------------------------------------------------------------
Type of initialization: random
-----------------------------------------------------------------------------------------------
Computing CPD
Final results
    Number of steps = 169
    Relative error = 1.3678985301695948e-09
    Accuracy =  100.0 %


Again, we can convert everything to dense format and compute the exactly error. This example is not so big that won't fit in the computer memory, but usually one wants to avoid dense representation.

In [7]:
# Generate the coordinate (dense) format of the approximation.
T_approx = tfx.cpd2tens(factors)

# Generate the dense format from the sparse representation.
T_dense = tfx.sparse2dense(data, idxs, dims)

# Compute the error.
np.linalg.norm(T_dense - T_approx)/np.linalg.norm(T_dense)

1.3816644777860003e-09